<a href="https://colab.research.google.com/github/Nohalyan/Projetppchem/blob/Lucas1/Notebook_WSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Water Solubility Predisction



## 1.1 Import Relevant Modules and Libraries

Let's first start by importing relevant modules and libraries needed for this project.


In [1]:
# Install all libraries
!pip install pathlib numpy pandas rdkit matplotlib scikit-learn lightgbm lazypredict tqdm

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.3 MB 812.7 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.3 MB 1.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.3 MB 3.5 MB/s eta 0:00:01
   ------------------------------------- -- 1.2/1.3 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 1.0/99.8 MB 31.4 MB/s eta 0:00:04
    --------------------------------------- 1.8/99.8 MB 23.1 MB/s eta 0:00:05
   - -------------------------------------- 2.6/99.8 MB 20.7 MB/s eta 0:00:05
   - -------------------------------------- 3.5/99.8 MB 18.7 MB/s eta 0:00:06
   - -------------------------------------- 4.3/99.8 MB 18.3 MB/s eta 0:00:06
   -- --------------

Importation of the differents packages function to names for their following using

In [4]:
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

from lightgbm import LGBMRegressor

# lazypredict helps to train 42 ML models with a single line of code ans find the best ML models
import lazypredict
from lazypredict.Supervised import LazyRegressor

from tqdm import tqdm

import pickle


# 2.1 Let's get the Solubility Data

First, we will get solubility data from gashawmg (source: https://github.com/gashawmg) and perform exploratory data analysis

In [5]:
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data_Solubility.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


Let's open the file

In [6]:
# Create a Path object for the current directory, in our case /content/
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

file_path = current_directory / "Data_Solubility.csv"

# Reading the contents of the file and check that the file exists
if file_path.exists():
    with file_path.open("r") as file:
        content = file.read()
#       print(content)
else:
    print("The file does not exist.")


Current Directory: C:\Users\venan\github\Projetppchem
The file does not exist.


The file use semicicolon as delimiter, so let's open the file and use semicicolon as delimiter:

In [7]:
# open a file containing descriptors and yield
data_solubility = pd.read_csv("/content/Data_Solubility.csv", delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Data_Solubility.csv'

Check the data see if it is what we want

In [ ]:
data_solubility.shape

In [ ]:
data_solubility.head()

Looks nice to me!

# 2.2 Data Cleaning

## 2.2.1 Remove NaN or null values

We wil start by removing non-numerical values and valeurs that are null:

In [ ]:
data_solubility.SMILES.isnull().sum()
data_solubility.dropna(inplace=True)
data_solubility.shape

As we can see, the shape is still the same, the data has already been cleaned of non-numerical and null values.
##2.2.2 Remove outliers

Then, we will remove outliers from the data. Using a boxplot, we can easely visualize outliers:


In [ ]:
sn.set_theme()
sn.displot(data=data_solubility, x="logS", binwidth=1)

Let's filter compounds that follow as close as normal distribution, let's say between -7.5 and 1.7:


In [ ]:
new_data_solubility = data_solubility[data_solubility.logS.apply(lambda x: x > -7.5 and x < 1.7)]

Let's generate an histogram to see the new data:

In [ ]:
sn.displot(data=new_data_solubility, x='logS', binwidth=1,kde=True)
new_data_solubility.shape

##2.2.3 Remove Duplicates

Then remove duplicate by generating canonical SMILES:

In [ ]:
# generate a canonical SMILES function
def canonical_SMILES(smiles):
    canon_smls = [Chem.CanonSmiles(smls) for smls in smiles]
    return canon_smls

In [ ]:
# Generate canonical Smiles using the function
canon_smiles = canonical_SMILES(new_data_solubility.SMILES)

# Replace SMILES column with canonical SMILES
new_data_solubility["SMILES"] = canon_smiles

# Create a list for duplicate smiles
duplicate_smiles = new_data_solubility[new_data_solubility['SMILES'].duplicated()]['SMILES'].values
len(duplicate_smiles)

As we can see, their are 6 duplicates, so we have to filter them and we can also sort them for better reading:

In [ ]:
new_data_solubility[new_data_solubility['SMILES'].isin(duplicate_smiles)].sort_values(by=['SMILES'])

Let's drop rows that contain duplicate SMILES and keep the first structure:

In [ ]:
data_solubility_cleaned = new_data_solubility.drop_duplicates(subset=['SMILES'], keep='first')
data_solubility_cleaned.shape
data_solubility_cleaned.head()

## 2.2.4 Filter training data

Now that we have a dataset, let's prepapare a test stet containing 100 drug-like compounds (source: https://github.com/PatWalters/solubility)

In [ ]:
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data_Drug_Like_Solubility.csv

In [ ]:
# Create a Path object for the current directory, in our case /content/
current_directory_dl = Path.cwd()
print("Current Directory:", current_directory.resolve())

file_path_dl = current_directory / "Data_Drug_Like_Solubility.csv"

# Reading the contents of the file and check that the file exists
if file_path.exists():
    with file_path.open("r") as file:
        content = file.read()
#        print(content)
else:
    print("The file does not exist.")


In [ ]:
data_dl = pd.read_csv("/content/Data_Drug_Like_Solubility.csv", delimiter=';')
data_dl.shape

In [ ]:
data_dl.head()

In [ ]:
# Generate canonical Smiles
canon_smiles = canonical_SMILES(data_dl.SMILES)

# Replace SMILES column wit Canonical SMILES
data_dl["SMILES"] = canon_smiles

# Create a list for duplicate smiles
duplicate_data_dl_smiles = data_dl[data_dl['SMILES'].duplicated()]['SMILES'].values
len(duplicate_data_dl_smiles)

In [ ]:
# Molecules used in training and test of the model
data_dl_SMILES = data_dl.SMILES.values

# Filter molecules that are not present in the test set
data_cleaned_final = data_solubility_cleaned[~data_solubility_cleaned['SMILES'].isin(data_dl_SMILES)]
print(f'Compounds present in training set:{len(data_solubility_cleaned) - len(data_cleaned_final)}')
data_cleaned_final.shape

In [ ]:
data_dl= data_dl[data_dl['LogS exp (mol/L)'].apply(lambda x: x > -7.5 and x < 1.7)]
data_dl

# 3. Calculation of RDkit Molecular Descriptors, which are molecular features

In [ ]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0]
                                    for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in tqdm(mols):
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(data_cleaned_final['SMILES'])

In [ ]:
df_descriptors = pd.DataFrame(Mol_descriptors, columns=desc_names)
df_descriptors.head()

In [ ]:
df_descriptors.shape

# 4. Split the chemicals for training and validation set

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(df_descriptors, data_cleaned_final.logS, test_size=0.1,random_state=42)

#Standardization of the features

custom_scaler = StandardScaler()
custom_scaler.fit(x_train)
x_train_scaled = custom_scaler.transform(x_train)
x_valid_scaled = custom_scaler.transform(x_valid)

#5. Select Machine Learning Models

Let's selct the best ML models for that. To do that, we will use the lazypredict librarie, in particular the LazyRegressor function to test 42 ML models:

In [ ]:
lregs = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None,random_state=42)
models, prediction_tests = lregs.fit(x_train_scaled, x_valid_scaled, y_train, y_valid)

In [ ]:
#The top three models
prediction_tests[:5]

#6. Fine-tuning of LGBMRegressor

We decided to take the LGBMRegressor model because the results generated by this model are comparable to the ExtraTreesRegressor model, but takes a lot less time than the extra-trees model.Let's performs a grid search using GridSearchCV from scikit-learn to find the best hyperparameters for a LightGBM regressor:

In [ ]:
#params = {'max_depth' : list(range(2, 32, 8)),
#          'n_estimators' : list(range(1, 1000, 100)),
#          'learning_rate' : list(np.arange(0.01, 1.02, 0.25))}
#
#grid_search = GridSearchCV(LGBMRegressor(random_state = 42),
#                            param_grid=params, cv=5, verbose=1)
#
#grid_search.fit(x_train, y_train)
#
#print("Optimized parameters for a LightGBM regressor can be: ", grid_search.best_params_)

We obtained:
* learning_rate: 0.01
* max_depth: 26
* n_estmitors: 901

Let's optimize again with new ranges for liste max_depth, n_estmitors and learning_rate to obtain even better parameters:


In [ ]:
#params_bst = {"max_depth" : list(range(20, 36, 4)),
#         "n_estimators" : list(range(850, 1200, 50)),
#         "learning_rate" : list(np.arange(0.01, 0.05, 0.01))}
#
#grid_search_bst = GridSearchCV(LGBMRegressor(random_state = 42),
#                                  param_grid=params_bst, cv=3, verbose=1)
#
#grid_search_bst.fit(x_train, y_train)
#print("The best parameters are: ", grid_search_bst.best_params_)

#7. LGBMRegressor model for training and test data
Let's test our model ont the training and test set with the best parameters found with the fine tunning:
* learning_rate: 0.04
* max_depth: 24
* n_estmitors: 1150




In [ ]:
model = LGBMRegressor(n_estimators = 1150,
                      max_depth = 24,
                      learning_rate = 0.04,
                      random_state= 42)

model.fit(x_train_scaled,y_train)
y_preds = model.predict(x_valid_scaled)

In [ ]:
# A plotting function
def plot_data(actual, predicted, title):

# model performance using RMSE
    rmse = np.sqrt(mean_squared_error(actual, predicted))

# R^2 (coefficient of determination) :
    R2 =r2_score(actual, predicted)
    plt.figure(figsize=(8,6))

# Plot the figure
    sn.regplot(x=predicted , y=actual,line_kws={"lw":2,
                                                "ls":"--",
                                                "color":"red",
                                                "alpha":0.7})
    plt.title(title, color="red")
    plt.xlabel("Predicted logS(mol/L)",
               color="blue")
    plt.xlim(-8,1)
    plt.ylabel("Experimental logS(mol/L)",
               color ="blue")


    plt.grid(alpha=0.3)
    R2 = mpatches.Patch(label="R2={:04.2f}".format(R2))
    rmse = mpatches.Patch(label="RMSE={:04.2f}".format(rmse))
    plt.legend(handles=[R2, rmse])

Let's plot the predicted logS of the validation set and see if our model works:

In [ ]:
sn.set_theme(style="whitegrid")
plot_data(y_valid,y_preds,"Validation data")

In [ ]:
#Calculate molecular descriptors for the test data or 98 compounds

Mol_descriptors_test , desc_names_test = RDkit_descriptors(data_dl["SMILES"])
data_dl_descriptors = pd.DataFrame(Mol_descriptors_test,columns=desc_names_test)

# Standard scaler - transform
x_scaled_test = custom_scaler.transform(data_dl_descriptors)

# Predict solubility of the test data
y_test_preds = model.predict(x_scaled_test)



In [ ]:
# Plotting testing set
sn.set_theme(style="whitegrid")
plot_data(data_dl["LogS exp (mol/L)"], y_test_preds,
           "Test data: Drug-like Molecules")

#8. Saving of the trained model and standard scaler

In [ ]:
with open("model_WSP.pkl","wb") as f:
    pickle.dump(model,f)

with open("scaler_WSP.pkl","wb") as f:
    pickle.dump(custom_scaler,f)